In [1]:
!pip install pymongo
import pymongo
cli = pymongo.MongoClient(host='34.212.59.71')

    100% |████████████████████████████████| 368kB 2.7MB/s ta 0:00:011
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
cli.database_names()

['admin', 'local', 'wiki_mongo_database', 'wiki_mongo_db']

In [4]:
wiki_mongo_database = cli.wiki_mongo_database
wiki_cat_collection = cli.wiki_mongo_database.wiki_cat_collection
wiki_cos_sim_collection = cli.wiki_mongo_database.wiki_cos_sim_collection

In [5]:
wiki_df = pd.DataFrame(list(cli.wiki_mongo_database.wiki_mongo_collection.find({})))
wiki_df.head(2)

,_id,category,clean_content,content,pageid,title
0,591b9c60ad28cdd96d814531,machine learning,approximate near neighbor search special case ...,(1+ε)-approximate nearest neighbor search is a...,32003319,(1+ε)-approximate nearest neighbor search
1,591b9c65ad28cdd96d814535,machine learning,adaline adaptive linear neuron later adaptive ...,ADALINE (Adaptive Linear Neuron or later Adapt...,8220913,ADALINE


In [6]:
wiki_cat_collection.drop()
for c in list(wiki_df['category'].unique()):
    text = ''
    for i in wiki_df['clean_content'][wiki_df['category']==c]:
        text += ' '+ i
    dict_ = {
            'category': c,
            'text': text
                }
    wiki_cat_collection.update_one(dict_,{'$set': dict_}, upsert=True)

In [7]:
wiki_cat_df = pd.DataFrame(list(cli.wiki_mongo_database.wiki_cat_collection.find({})))
len(wiki_cat_df['text'][wiki_cat_df['category']=='machine learning'][0])

4549932

In [8]:
wiki_cat_df[['category','text']].head()

,category,text
0,machine learning,approximate near neighbor search special case...
1,Microsoft Office,analyse statistical analysis add microsoft ex...
2,Geographic information systems,world atlas virtual globe program develop cos...
3,Petroleum,avenue americas know news corp building inter...
4,Yoga,adho mukha sana adho mukha shvanasana ipa muk...


In [9]:
from spacy.en import English
nlp = English()

In [10]:
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
cat_vecs = np.array([nlp(i).vector for i in wiki_cat_df['text']])

In [12]:
content_vecs = np.array([nlp(i).vector for i in wiki_df['clean_content']])

In [13]:
import pickle
!pip install redis
import redis
redis_ip = '34.212.59.71'
r = redis.StrictRedis(redis_ip)
nlp_cat_vecs = pickle.dumps(cat_vecs)
r.set('nlp_cat_vecs', nlp_cat_vecs)
nlp_content_vecs = pickle.dumps(content_vecs)
r.set('nlp_content_vecs', nlp_content_vecs)

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


True

In [14]:
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
import redis
redis_ip = '34.212.59.71'
content_vecs = pickle.loads(r.get('nlp_content_vecs'))
cat_vecs = pickle.loads(r.get('nlp_cat_vecs'))
def Similarity_score(text):
    wiki_cat_df = pd.DataFrame(list(cli.wiki_mongo_database.wiki_cat_collection.find({})))
    #cat_vecs = np.array([nlp(i).vector for i in wiki_cat_df['text']])
    r = redis.StrictRedis(redis_ip)
    cat_vecs = pickle.loads(r.get('nlp_cat_vecs'))
    similarity_score={}
    for j in range(len(cat_vecs)):
        similarity = cosine_similarity(cat_vecs[j].reshape(1,-1),nlp(text).vector.reshape(1,-1))
        similarity_score[(wiki_cat_df['category'][j])] = round(similarity[0][0],3)
        cs_s_df = pd.DataFrame.from_dict(similarity_score,orient='index')
        cs_s_df.columns = ['score']
        cs_s_df = cs_s_df.sort_values('score',ascending=False)
    return cs_s_df.head(3)

In [18]:
Similarity_score('lake')

,score
Petroleum,0.418
Taiwan,0.402
Yoga,0.379


In [19]:
wiki_cos_sim_collection.drop()
dict_cos_sim={}
import redis
redis_ip = '34.210.97.79'
content_vecs = pickle.loads(r.get('nlp_content_vecs'))
cat_vecs = pickle.loads(r.get('nlp_cat_vecs'))

for i in range(len(content_vecs)):
    dict_cos_sim['title']=  wiki_df['title'][i]
    dict_cos_sim['category']= wiki_df['category'][i]
    for j in range(len(cat_vecs)):
        similarity = cosine_similarity(cat_vecs[j].reshape(1,-1),content_vecs[i].reshape(1,-1))        
        dict_cos_sim['{}'.format(wiki_cat_df['category'][j])]= round(similarity[0][0],3)
    wiki_cos_sim_collection.update_one(dict_cos_sim,{'$set': dict_cos_sim}, upsert=True)

In [20]:
cli.wiki_mongo_database.collection_names()

['wiki_cos_sim_collection', 'wiki_mongo_collection', 'wiki_cat_collection']

In [21]:
list(wiki_df['category'].unique())

['machine learning',
 'Microsoft Office',
 'Geographic information systems',
 'Petroleum',
 'Yoga',
 'Business intelligence',
 'Taiwan',
 'Statistical_methods']

In [22]:
cos_df = pd.DataFrame(list(cli.wiki_mongo_database.wiki_cos_sim_collection.find({})))
cos_df = cos_df[['category','title','machine learning',\
                 'Microsoft Office',\
                 'Geographic information systems',\
                 'Petroleum',\
                 'Yoga',\
                 'Business intelligence',\
                 'Taiwan',\
                 'Statistical_methods']]

In [ ]:
cos_df.iloc[3, 2:-1].sort_values(ascending=False).index[0]

'Yoga'

In [ ]:
cos_df['predicted_cat'] = 0
for i in range(len(cos_df)):
    cos_df['predicted_cat'][i] = cos_df.iloc[i, 2:-1].sort_values(ascending=False).index[0]   

/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.5/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
cos_df

In [ ]:
print('pridiction score with Cosine Similarity:', 
      sum(cos_df['category'] == cos_df['predicted_cat'])/len(cos_df))